### IMPORT & FIX

In [1]:
import os
import time
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from tempfile import TemporaryDirectory
from PIL import ImageFile
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.tensorboard import SummaryWriter

2025-05-27 10:32:15.255542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748341935.473713      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748341935.535519      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Fix truncated images - For this dataset
ImageFile.LOAD_TRUNCATED_IMAGES = True

### PROCESSING DATA

In [3]:
# Transform data
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.4502, 0.4242, 0.3926], [0.2224, 0.2084, 0.2035])
    ]), 
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.4502, 0.4242, 0.3926], [0.2224, 0.2084, 0.2035])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.4502, 0.4242, 0.3926], [0.2224, 0.2084, 0.2035])
    ])
}

In [4]:
# Declare directories
data_dir = "/kaggle/input/ai-and-real-images"
folders = ["train", "val", "test"]

In [5]:
# Load image datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in folders}

# Loader for mini batch
loader = {x: DataLoader(image_datasets[x], batch_size = 32, shuffle = True, num_workers = 4) for x in folders}

In [6]:
# Dataset sizes and class names
dataset_sizes = {x: len(image_datasets[x]) for x in folders}
class_names = image_datasets["train"].classes

### CONFIGURE & TRAIN MODEL

In [7]:
# Get torch device
device = torch.device("cuda" if torch.accelerator.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [8]:
# Make directory for saving model and stats
os.mkdir("/kaggle/working/runs")

In [9]:
# Use tensorboard to store stats and make a directory to save
os.mkdir("/kaggle/working/runs/tensorboard")
writer = SummaryWriter(log_dir='/kaggle/working/runs/tensorboard')

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch+1}/{num_epochs}')
            print('-' * 10)

            # Each epoch: Train phase and Val phase
            for phase in ["train", "test"]:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data
                for inputs, labels in loader[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # Zero gradients
                    optimizer.zero_grad()

                    # Forward
                    with torch.set_grad_enabled(phase == 'train'): # grad_enable: True (Train), False (Val)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # Backward + Optimize
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # Stats
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]
                
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                writer.add_scalar(f'{phase}/Loss', epoch_loss, epoch)
                writer.add_scalar(f'{phase}/Accuracy', epoch_acc, epoch)

                # Deep copy the model
                if phase == 'test' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return model

In [11]:
# Download the pretrain model
model_ft = models.convnext_tiny(weights='IMAGENET1K_V1')

# Change the classifier layer
model_ft.classifier[2] = torch.nn.Linear(model_ft.classifier[2].in_features, len(class_names))

# Check if it multigpu or not to using DataParallel (MultiGPU training)
if torch.cuda.device_count() > 1:
    model_ft = torch.nn.DataParallel(model_ft)
model_ft.to(device)

### Output is a architecture of model after changing the out_features

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:00<00:00, 199MB/s] 


DataParallel(
  (module): ConvNeXt(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
      )
      (1): Sequential(
        (0): CNBlock(
          (block): Sequential(
            (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (1): Permute()
            (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
            (3): Linear(in_features=96, out_features=384, bias=True)
            (4): GELU(approximate='none')
            (5): Linear(in_features=384, out_features=96, bias=True)
            (6): Permute()
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): CNBlock(
          (block): Sequential(
            (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (1): Permute()
            (2): LayerNorm((96,)

In [12]:
# Cross Entropy
criterion = torch.nn.CrossEntropyLoss()

# SGD optimizer - lr e-3, momentum 0.9
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Step LR - step_size 1, gamma 0.9
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.9)

In [13]:
# TRAIN MODEL
model_conv = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 1/25
----------


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.4358 Acc: 0.7964


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


test Loss: 0.3574 Acc: 0.8385

Epoch 2/25
----------


...

test Loss: 0.1535 Acc: 0.9423

Epoch 25/25
----------


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.0858 Acc: 0.9681


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


test Loss: 0.1525 Acc: 0.9435

Training complete in 309m 47s
Best val Acc: 0.943500


### SAVE & PLOT MODEL

In [14]:
# Save the model
os.mkdir("/kaggle/working/runs/model")
torch.save(model_conv.state_dict(), "/kaggle/working/runs/model/best_model.pt")

In [15]:
# Run model with test set
model_conv.eval() # Eval mode

y_pred = []
y_true = []

for inputs, labels in loader["val"]:
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    # Forward - Grad disable
    with torch.set_grad_enabled(False):
        outputs = model_conv(inputs)
        _, preds = torch.max(outputs, 1)
    
    y_pred.append(preds)
    y_true.append(labels)

# Concatenates all the mini-batch
y_pred = torch.cat(tuple(y_pred))
y_true = torch.cat(tuple(y_true))

# Confusion matrix
cfs_mtx = confusion_matrix(y_true.cpu(), y_pred.cpu())
display = ConfusionMatrixDisplay(cfs_mtx)

display.plot()

writer.add_figure("Confusion Matrix (Test set)", display.figure_)